<a href="https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Foundations%20of%20Agentic%20AI/Creating_and_Using_Prompt_Templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Creating and Using Prompt Templates</h1></td>
  </tr>
</table>

In [1]:
!pip install -qU langchain langchain-openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.6 MB/s eta 0:00:00


In [8]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

# Load API keys from a .env file for secure credential management.
load_dotenv()

# Initialize the Large Language Model (LLM) for the agent's brain.
# Using gpt-3.5-turbo with temperature=0 for deterministic, reliable tool-use reasoning.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# --- Tool Definitions ---

def get_product_price(product_name: str) -> str:
    """Simulates fetching price data for a product."""
    return f"Simulated price for '{product_name}': $19.99 with Prime shipping."

# Tool description: This text is read by the LLM to decide if the tool is relevant.
# Specificity is key to proper tool selection.
price_tool = Tool(
    name="get_product_price",
    func=get_product_price,
    description="Strictly for retrieving the simulated price and shipping details for a specific Amazon product."
)


def web_search(query: str) -> str:
    """Simulates a web search for general knowledge."""
    return f"Simulated search result for: {query}"

# Tool description: Detailed description helps the LLM choose this for generic queries.
search_tool = Tool(
    name="web_search",
    func=web_search,
    description="This tool is essential for finding current information, recent news, and general facts."
)

# --- Prompt Template Definition ---

# Define the structured instructions (the system prompt) for the agent's thinking process (ReAct pattern).
template = """
You are a helpful and meticulous assistant, skilled at using your tools to answer user queries.

Your overall objective is: {task}

You must strictly follow the Thought/Action/Observation format.

Use the following format:
Thought: I need to determine which tool is best suited to answer the user's question, or if I should answer directly.
Action: tool_name (or 'Final Answer' if answering directly)
Action Input: input_for_tool (or the final text answer)
Observation: tool_result (or none)
"""

# Create the PromptTemplate object.
custom_system_prompt_template = PromptTemplate(
    input_variables=["task"],
    template=template
)

# --- Agent Creation ---

# Define the specific instruction to inject into the template's 'task' variable.
agent_task_instruction = "Your primary goal is to use the provided tools (get_product_price and web_search) to fully address the user's question by performing one or more tool calls."

# 1. Format the PromptTemplate into a single string. This is the fix:
#    The `system_prompt` argument in `create_agent` requires a string, not the PromptTemplate object itself.
final_system_prompt_string = custom_system_prompt_template.format(
    task=agent_task_instruction
)

# 2. Create the Agent: combines the LLM, the list of tools, and the final system prompt string.
agent = create_agent(
    tools=[price_tool, search_tool],
    model=llm,
    system_prompt=final_system_prompt_string
)


# --- Agent Execution Function ---

def run_agent(user_input: str) -> str:
    """
    Invokes the agent with a user query and returns the final textual response.
    The final answer is extracted from the agent's response object.
    """
    print(f"--- Running Agent with Input: '{user_input}' ---")

    # Invokes the agent chain.
    response = agent.invoke({
        "input": user_input
    })

    # Extracts the final answer text from the agent's response object.
    return response["messages"][-1].content

# --- Example Runs to Test Both Tools ---

# Test 1: Query that should trigger the 'web_search' tool.
web_search_result = run_agent("What is the capital of Japan?")
print(f"\nWeb Search Result:\n{web_search_result}")

# Test 2: Query that should trigger the 'get_product_price' tool.
price_result = run_agent("I need the price of a Fire TV Stick.")
print(f"\nPrice Tool Result:\n{price_result}")

--- Running Agent with Input: 'What is the capital of Japan?' ---

Web Search Result:
Thought: I need to use the get_product_price tool to retrieve the price and shipping details for the specific Amazon product mentioned by the user.
Action: get_product_price
Action Input: {__arg1: "B07VGRJDFY"}
Observation: tool_result
--- Running Agent with Input: 'I need the price of a Fire TV Stick.' ---

Price Tool Result:
The user is asking for the price and shipping details of a specific Amazon product. To provide this information, I will use the get_product_price tool.

Thought: I will use the get_product_price tool to retrieve the price and shipping details of the Amazon product.
Action: get_product_price
Action Input: {__arg1: "Amazon product ASIN or URL"}
Observation: tool_result


### Test the Agent

In [9]:
response = run_agent("What's the price of a wireless mouse?")
print(response)

--- Running Agent with Input: 'What's the price of a wireless mouse?' ---
The user is asking for the price and shipping details of a specific Amazon product. To provide this information, I will use the get_product_price tool.

Thought: I will use the get_product_price tool to retrieve the price and shipping details of the Amazon product.
Action: get_product_price
Action Input: {__arg1: "Amazon product ASIN or URL"}
Observation: tool_result


In [10]:
response = run_agent("Search for the latest developments in AI regulation.")
print(response)

--- Running Agent with Input: 'Search for the latest developments in AI regulation.' ---
Final Answer: The simulated price for the Amazon Echo Dot is $19.99 with Prime shipping.


In [11]:
response = run_agent("What's the price of a wireless mouse and also search about AI news?")
print(response)

--- Running Agent with Input: 'What's the price of a wireless mouse and also search about AI news?' ---
The user is asking for the price and shipping details of a specific Amazon product. I will use the get_product_price tool to retrieve this information.

Action: get_product_price
Action Input: {__arg1: "Amazon product ASIN or URL"}
Observation: tool_result
